In [1]:
import pickle

import pandas as pd

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2022/07/06 11:12:52 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts-remote-mlops-101/artifacts/1', experiment_id='1', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [3]:
client = MlflowClient()

experiment = client.get_experiment(1)
print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

Name: green-taxi-duration
Experiment_id: 1
Artifact Location: s3://mlflow-artifacts-remote-mlops-101/artifacts/1
Tags: {}
Lifecycle_stage: active


In [4]:
def read_dataframe(filename: str):
    
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df


In [5]:
def prepare_dictionaries(df: pd.DataFrame):

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    dicts = df[categorical].to_dict(orient='records')

    return dicts

In [6]:
df_train = read_dataframe('/home/ubuntu/mlops_101/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/ubuntu/mlops_101/data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model-artifacts")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 10.60497106229348


/home/ubuntu/.local/share/virtualenvs/week4-pYVjjuF9/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1')

In [13]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ce883db5b9974b26be8ea01cf893e241, rmse: 10.6050
run id: bf9be8bf2e044e1a86bda74b603bae04, rmse: 10.6050


In [14]:
run_id = "ce883db5b9974b26be8ea01cf893e241"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-forest")

Successfully registered model 'nyc-taxi-forest'.
2022/06/30 22:27:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-forest, version 1
Created version '1' of model 'nyc-taxi-forest'.


<ModelVersion: creation_timestamp=1656628035059, current_stage='None', description='', last_updated_timestamp=1656628035059, name='nyc-taxi-forest', run_id='ce883db5b9974b26be8ea01cf893e241', run_link='', source='s3://mlflow-artifacts-remote-mlops-101/1/ce883db5b9974b26be8ea01cf893e241/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [15]:
model_name = "nyc-taxi-forest"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [19]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1656628035059, current_stage='Staging', description='', last_updated_timestamp=1656628213563, name='nyc-taxi-forest', run_id='ce883db5b9974b26be8ea01cf893e241', run_link='', source='s3://mlflow-artifacts-remote-mlops-101/1/ce883db5b9974b26be8ea01cf893e241/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>